Парсинг всех медий в виде {name, country, bias, factuality}

Второй скрипт (после этого) добавляет link по полю name

In [ ]:
from bs4 import BeautifulSoup
import requests
import json

def main(i):
    url = f"https://mediabiasfactcheck.com/filtered-search/?pg={i}"  # Замените на нужную ссылку

    response = requests.get(url)
    html = response.text

    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table', class_="mbfc-table")
    rows = table.find_all('tr')

    for row in rows:
        cells = row.find_all('td')
        cell_values = [cell.get_text(strip=True) for cell in cells]
        if cell_values:
            print(cell_values[0], cell_values[1], cell_values[2], cell_values[3])
            data = {
                "name": cell_values[0],
                "country": cell_values[3],
                "bias": cell_values[1],
                "factuality": cell_values[2]
            }
            cell_values[0] = (cell_values[0]).replace("-", '%').replace('\\','!').replace(' ', '_').replace('/', '$')
            with open(f"labels/{cell_values[0]}.json", "w", encoding="utf-8") as file:
                json.dump(data, file, indent=4)

for i in range(50, 69):
    main(i)



Парсинг всех ссылок на медии, в виде пары (name, link)

Файл links.json просто скопирован из архива на mediabiasfactcheck.com

In [ ]:
import json
import os

try:
    with open("links.json", "r", encoding="utf-8") as f:
        data = json.load(f)
except json.decoder.JSONDecodeError as e:
    error_position = e.pos
    error_character = e.doc[e.pos]
    print(f"Проблемный символ: '{error_character}' (позиция {error_position})")


with open("links_.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=4)

Сборка полноценного json файла с полем link

In [ ]:
import json
import os

count = 0

# соединение со старым датасетом, убирая повторы
dir1 = "labels"
dir2 = "links"
for file1 in os.listdir(dir1):
    for file2 in os.listdir(dir2):

            if(file1 == file2):
                try:

                    count += 1
                    with open(os.path.join(dir1, file1), "r", encoding="utf-8") as f1:
                        data1 = json.load(f1)
                    with open(os.path.join(dir2, file2), "r", encoding="utf-8") as f2:
                        data2 = json.load(f2)
                    data = {
                        "name": data1["name"],
                        "country": data1["country"],
                        "bias": data1['bias'],
                        "factuality": data1['factuality'],
                        "link": data2['link']
                    }
                    name = file1
                    with open(f"no_repeat/{name}", "w", encoding="utf-8") as f:
                        json.dump(data, f, indent=4)
                except json.JSONDecodeError:
                    print(f"ERR WITH {file1} and {file2}")


print(count)


In [ ]:
# парсинг контента с первой страницы сайта медии (в среднем 10-15 статей)

import os
import requests
from bs4 import BeautifulSoup
import json

dataset_dir = 'no_repeat'
#dataset_dir = 'test'
json_files = [f for f in os.listdir(dataset_dir) if f.endswith('.json')]

def get_data(url):
    try:
        response = requests.get(url, timeout=3)
        text = response.text
        content = BeautifulSoup(text, 'html.parser').get_text()
        return content
    except Exception as e:
        print(f"Bad link in {url}")
        with open("log.txt", 'a', encoding='utf-8') as f:
             f.write(f"Bad link in {url}")
small_control = []
for json_file in json_files:
    with open(os.path.join(dataset_dir, json_file), 'r', encoding='utf-8') as f:
        data = json.load(f)
        url = (data['link']).replace("https://", "")
        url = "https://" + url
        content = get_data(url)
        if content:
            data['content'] = content

            #(если текст слишком маленький - удаляем)
            # это значит что автоматизировать парсинг не получается
            # тк либо сайт медии заблокирован
            # либо выскакивает соглашение на обработку данных и cookie
            # и парсится текст либо ошибки либо этого соглашения
            # такой мусорный парсинг не нужен. Удаляем

            if len(content) < 150:
                print(f"small content in {json_file}")
                small_control.append(json_file)

        else:
            data['content'] = ""

    with open(os.path.join(dataset_dir, json_file), 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

    print(f"Content added to {json_file}.")

print(small_control)
print(len(small_control))

Пост обработка.

Добавление numeric к bias и factuality

In [ ]:
import json
import os

directory = 'gold'
def add_num_once(filep):
    file = os.path.join(filep)
    # Открываем файл и загружаем его содержимое в словарь
    with open(file, 'r', encoding="utf-8") as f:
        data = json.load(f)

    # Добавляем числовые поля к существующим текстовым полям
    try:
        bias_text = data['bias']

        if bias_text == 'Left':
            data['bias (numeric)'] = 0
        elif bias_text == 'Left-Center':
            data['bias (numeric)'] = 1
        elif bias_text == 'Center' or bias_text == 'Least Biased':
            data['bias (numeric)'] = 2
        elif bias_text == 'Right-Center':
            data['bias (numeric)'] = 3
        elif bias_text == 'Right':
            data['bias (numeric)'] = 4
        else :
            data['bias (numeric)'] = -1
    except IndexError:
        print(file)

    try:
        factuality_text = data['factuality']
        if factuality_text == 'Very High':
            data['factuality (numeric)'] = 0
        elif factuality_text == 'High':
            data['factuality (numeric)'] = 1
        elif factuality_text == 'Mostly Factual':
            data['factuality (numeric)'] = 2
        elif factuality_text == 'Mixed':
            data['factuality (numeric)'] = 3
        elif factuality_text == 'Low':
            data['factuality (numeric)'] = 4
        elif factuality_text == 'Very Low':
            data['factuality (numeric)'] = 5
        else:
            data['factuality (numeric)'] = -1
    except IndexError:
        print(file)
   # data[0]['Link'] = 'https://' + data[0]['Link']
    # Сохраняем измененный словарь обратно в JSON файл
    with open(file, 'w') as f:
        json.dump(data, f, indent=4)


for filename in os.listdir(directory):
    # Объединяем имя файла и путь к директории
    filepath = os.path.join(directory, filename)
    # Проверяем, является ли файл JSON-файлом
    if filepath.endswith('.json'):
        add_num_once(filepath)

Рандомизация имени и поля id

In [ ]:
import os
import random
import string
import json

# Указываем путь к папке, где находятся файлы
folder_path = 'dir'

# Проходимся по всем файлам в папке
for filename in os.listdir(folder_path):
    old_file_name = filename

    # Генерируем случайный id
    new_file_name = ''.join(random.choices(string.ascii_uppercase + string.digits, k=8)) + '.json'
    new_file_path = os.path.join(folder_path, new_file_name)

    # Переименовываем файл
    os.rename(folder_path + '/' + old_file_name, new_file_path)

    # Обновляем id в файле
    with open(new_file_path, 'r+', encoding='utf-8') as f:
        data = json.load(f)
        data['id'] = new_file_name[:-5]
        f.seek(0)
        json.dump(data, f, indent=2)
        f.truncate()

Приведение файла к нужному виду.

Настройка порядка полей для удобной проверки качества парсинга

In [ ]:
import os
import json
import shutil
import random
import string


# Путь к директории с JSON-файлами
path = 'gold'

# Проходимся по всем файлам в директории
for filename in os.listdir(path):
    # Проверяем, что файл имеет расширение .json
    if filename.endswith(".json"):
        # Считываем содержимое файла
        with open(os.path.join(path, filename), "r", encoding='utf-8') as f:
            content = json.load(f)
        data = {
            "id": ''.join(random.choices(string.ascii_letters + string.digits, k=8)),
            "name": content["name"],
            "country": content["country"],
            "bias": content["bias"],
            "bias (numeric)": content["bias (numeric)"],
            "factuality": content["factuality"],
            "factuality (numeric)": content["factuality (numeric)"],
            "link": content["link"],
            "content": content["content"]
        }
        with open(os.path.join(path, filename), "w", encoding='utf-8') as f:
            json.dump(data, f, indent=4)